## JEM207 Data Processing in Python - Midterm
[Daniel Brosko](mailto:89227653@fsv.cuni.cz), April 5th, 2022

#### Just testing

In [1]:
# Just testing
import numpy as np

In [2]:
a = np.array([0, 1, 2, 3, 4])

print(a)

a.shape

len(a)

[0 1 2 3 4]


5

## Task 1

## Task 2

## Task 3